# Classifying MNIST digits using Logistic Regression

This notebook closely follows this tutorial: http://deeplearning.net/tutorial/logreg.html

The MNIST digits dataset is separated in training, verification and test subsets. A logistic regression classifier is trained with the following model:
```
prediction = argmax_i(softmax_i(Wx + b))
```
where
* `prediction` is the predicted class (digit) for the input `x`
* `W` is a weight matrix
* `b` is a bias vector
* `i` is one of the possible classes (digits)
The classifier uses the negative log likelihood (NLL) loss function:
```
l = -log(P(Y=y(i)|x(i),W,b))
```
where
* `x(i)` is an input vector that should be classified as class `i`
* `y(i)` is a prediction for class `i`

Useful flags:
```
THEANO_FLAGS=floatX=float64,device=cuda,traceback.limit=20
```
use `device=cuda` or `device=cpu`

Imports section

In [1]:
import pickle
import gzip
import os
import timeit

import numpy

import theano
import theano.tensor as T

Some global constants

In [2]:
# The MNIST dataset - loaded in suitable python format and pickled (see the tutorial)
DATASET = "E:/ml/datasets/mnist.pkl.gz"

# The save location for the best trained model
SAVED_MODEL = "E:/ml/mnist/my_best_model.pkl"

Definition of a helper function that reads a dataset from gziped pickle file and loads it into shared Theano variables. The idea is to have the whole dataset copied into the GPU's memory in a single operation rather than copying it in chucks on each training step. The slices of the data that are needed for a given operation (training/verification/test step) are accessed via indices later.

In [3]:
def load_data(dataset):
    data_dir, data_file = os.path.split(dataset)
    if data_dir == "" and not os.path.isfile(dataset):
        raise ValueError('Incorrect dataset path', ('dataset', dataset))
    print('... loading data')
    with gzip.open(dataset, 'rb') as f:
            train_set, validate_set, test_set = pickle.load(f, encoding='latin1')
    def shared_dataset(data_xy, borrow=True):
        data_x, data_y = data_xy
        shared_x = theano.shared(
            numpy.asarray(data_x, dtype=theano.config.floatX),
            borrow=borrow)
        # since we intend to keep this in the GPU's memory, we need to store it as a float and not an int
        shared_y = theano.shared(
            numpy.asarray(data_y, dtype=theano.config.floatX),
            borrow=borrow)
        # however, we cast it to int so we don't have to manage float pointing numbers when indexing the prediction classes
        return shared_x, T.cast(shared_y, 'int32')

    test_set_x, test_set_y = shared_dataset(test_set)
    validate_set_x, validate_set_y = shared_dataset(validate_set)
    train_set_x, train_set_y = shared_dataset(train_set)

    rval = [(train_set_x, train_set_y),
            (validate_set_x, validate_set_y),
            (test_set_x, test_set_y)]
    return rval

Definition of a LogisticRegression class. This will hold all the parameters (W and b) of our classifier. It also defines the loss function (negative_log_likelihood) for the classifier as well as a helper zero-one loss function (errors) that is used to validate the occuracy of the model during training. A trained model can be pickled so it can be later loaded and used out of the box.

In [4]:
class LogisticRegression(object):
    def __init__(self, x, n_in, n_out):
        """
        x: tensor, the input data for this LogisticRegression. This is a (micro)batch of one or more datapoints from th dataset
        n_in: integer, number of inputs (image width * height)
        n_out: integer, number of outputs (classes - 10 for digits)
        """
        
        # W and b are shared so the progress between iterations can be preserved
        self.W = theano.shared(
            value=numpy.zeros(
                (n_in, n_out),
                dtype=theano.config.floatX
            ),
            name='W',
            borrow=True
        )
        self.b = theano.shared(
            value=numpy.zeros(
                (n_out,),
                dtype=theano.config.floatX
            ),
            name='b',
            borrow=True
        )
        
        # define the model of the classifier
        self.p_y_given_x = T.nnet.softmax(T.dot(x, self.W) + self.b)
        self.y_pred = T.argmax(self.p_y_given_x, axis=1)
        
        self.params = [self.W, self.b]
        self.x = x
        
    # define the loss function of the model   
    def negative_log_likelihood(self, y):
        return -T.mean(T.log(self.p_y_given_x)[T.arange(y.shape[0]), y])
    
    # define a helper zero-one loss function for the model that can be used during testing/validation of the model
    def errors(self, y):
        return T.mean(T.neq(self.y_pred, y))

This is the heart of the algorithm - the microbatsches stohastic gradient descend (MSGD). Here the data is loaded and partitioned in microbatches. On each training step of the classifier, one whole batch of data is used. After all batches are used, we start all over and we count the next epoch. Classifier params are persisted between steps and epochs.

To try to prevent overfitting, an early exit will be triggered in the training algorithm in case the occuracy of the classifier does not improve in a number of steps.

In [5]:
def msgd_optimization_mnist(learning_rate=0.13,
                           n_epochs=1000,
                           dataset=DATASET,
                           batch_size=600):
    # load the dataset and split it in sets
    datasets = load_data(dataset)
    train_set_x, train_set_y = datasets[0]
    validate_set_x, validate_set_y = datasets[1]
    test_set_x, test_set_y = datasets[2]

    # Calculate the sizes of the batches
    n_train_batches = train_set_x.get_value(borrow=True).shape[0] // batch_size
    n_validate_batches = validate_set_x.get_value(borrow=True).shape[0] // batch_size
    n_test_batches = test_set_x.get_value(borrow=True).shape[0] // batch_size
    
    #############
    # Build model
    #############
    print('... building the model')
    
    # everything in this section is interpreted as symbolic assignments. Nothing is evaluated until a function is called
    
    index = T.lscalar() # the training batch index
    x = T.matrix('x') # the input. It is a matrix because multiple input vectors are processed at the same time
    y = T.ivector('y') # the labels for each input vector
    
    # instantiate the classifier and define a cost equal to the loss
    classifier = LogisticRegression(x=x, n_in=28*28, n_out=10)
    cost = classifier.negative_log_likelihood(y)
    
    # define the gradients of the classifier's parameters
    grad_W = T.grad(cost=cost, wrt=classifier.W)
    grad_b = T.grad(cost=cost, wrt=classifier.b)
    
    # gradient descent - on each step we are going to update W and b in proportion to their corresponding gradients
    updates = [
        (classifier.W, classifier.W - learning_rate * grad_W),
        (classifier.b, classifier.b - learning_rate * grad_b)
    ]
    
    # a helpper function that will be used to feed given batches of the datasets to a function
    def givens(dataset_x, dataset_y):
        return {
            x: dataset_x[index * batch_size : (index + 1) * batch_size],
            y: dataset_y[index * batch_size : (index + 1) * batch_size]
        }
    
    # Define the function that performs a training step
    train_model = theano.function(
        inputs=[index], # takes the batch index as input
        outputs=cost, # returns the cost as output
        updates=updates, # updates the model on each step
        givens=givens(train_set_x, train_set_y) # takes batches from the training sets
    )
    # Define a function that validates the occurracy of the model during training
    validate_model = theano.function(
        inputs=[index], # takes a batch index as input
        outputs=classifier.errors(y), # outputs the ratio of wrong predictions
        givens=givens(validate_set_x, validate_set_y) # takes batches from the validation sets
    )
    # Define a function that tests an already trained model
    test_model = theano.function(
        inputs=[index], # takes a batch index as input
        outputs=classifier.errors(y), # outputs the ratio of wrong predictions
        givens=givens(test_set_x, test_set_y) # takes batches from the training sets
    )
    
    #############
    # Train model
    #############
    print('... training the model')
    
    # In this section the model is actually trained, validated and tested.
    
    patience = 5000 # for early stopping, how many iterations to take at maximum
    patience_mult = 2 # when new best model is found, this much more iterations will be trained
    improvement_thresh = 0.995 # defines what is a "better model"
    validation_freq = min(n_train_batches, patience // 2) # number of iterations between validations

    best_validation_loss = numpy.inf
    test_score = 0.

    done_looping = False # for early stopping
    epoch = 0
    start_time = timeit.default_timer()
    
    while (epoch < n_epochs) and (not done_looping):
        epoch = epoch + 1
        for minibatch_index in range(n_train_batches):
            # train
            minibatch_avg_cost = train_model(minibatch_index)
            
            # do validation with all validation samples
            iter_no = (epoch - 1) * n_train_batches + minibatch_index
            if (iter_no + 1) % validation_freq == 0:
                # calculate the mean loss over all validation samples
                validation_loss = numpy.mean(
                    [validate_model(i) for i in range(n_validate_batches)]
                )
                print(
                    'epoch %i, minibatch %i/%i, validation error %f %%' %
                    (
                        epoch,
                        minibatch_index + 1,
                        n_train_batches,
                        validation_loss * 100.
                    )
                )
                
                if validation_loss < best_validation_loss:
                    # increase patience
                    if validation_loss < improvement_thresh * best_validation_loss:
                        patience = max(patience, iter_no * patience_mult)
                    best_validation_loss = validation_loss
                    
                    # calculate test score using all test samples
                    test_score = 1. - numpy.mean(
                        [test_model(i) for i in range(n_test_batches)]
                    )
                    print(
                        ' * epoch %i, minibatch %i/%i, test error of best model %f %%' %
                        (
                            epoch,
                            minibatch_index + 1,
                            n_train_batches,
                            test_score * 100.
                        )
                    )
                    
                    # pickle the best model for later use
                    with open(SAVED_MODEL, 'wb') as f:
                        pickle.dump(classifier, f)
            
            # early exit
            if iter_no >= patience:
                done_looping = True
                break
    
    end_time = timeit.default_timer()
    print(
        'Optimization complete with best validation score of %f %%, with test performance %f %%'
        % (
            best_validation_loss * 100.,
            test_score * 100.
        )
    )
    print(
        'The code run for %d epochs, with %f epochs/sec'
        % (
            epoch,
            1. * epoch / (end_time - start_time)
        )
    )
    print(('The notebook ran for %.1fs' % ((end_time - start_time))))

A helper function that can use a pre-trained model to predict the classes for some input samples

In [6]:
def predict(saved_model, x):
    """
    saved_model: string, location of the saved model
    x: tensor, input vectors to be classified concatenated into a tensor
    """
    with open(saved_model, 'rb') as f:
        classifier = pickle.load(f)
    
    prediction = theano.function(
        inputs=[classifier.x],
        outputs=classifier.y_pred
    )
    
    return prediction(x)

## Main entrypoint

This trains the model, saves the trained classifier and tests it with the test dataset.

In [7]:
msgd_optimization_mnist(dataset=DATASET)

... loading data
... building the model
... training the model
epoch 1, minibatch 83/83, validation error 12.458333 %
 * epoch 1, minibatch 83/83, test error of best model 87.625000 %
epoch 2, minibatch 83/83, validation error 11.010417 %
 * epoch 2, minibatch 83/83, test error of best model 89.041667 %
epoch 3, minibatch 83/83, validation error 10.312500 %
 * epoch 3, minibatch 83/83, test error of best model 89.687500 %
epoch 4, minibatch 83/83, validation error 9.875000 %
 * epoch 4, minibatch 83/83, test error of best model 90.166667 %
epoch 5, minibatch 83/83, validation error 9.562500 %
 * epoch 5, minibatch 83/83, test error of best model 90.520833 %
epoch 6, minibatch 83/83, validation error 9.322917 %
 * epoch 6, minibatch 83/83, test error of best model 90.708333 %
epoch 7, minibatch 83/83, validation error 9.187500 %
 * epoch 7, minibatch 83/83, test error of best model 91.000000 %
epoch 8, minibatch 83/83, validation error 8.989583 %
 * epoch 8, minibatch 83/83, test error 

## Pre-trained entrypoint

This uses a pre-trained model to predict the classes of the test samples from a dataset.

In [8]:
datasets = load_data(DATASET)
test_set_x, test_set_y = datasets[2]

get_labels = theano.function(
    inputs=[],
    outputs=test_set_y
)

k = 10
prediction = predict(SAVED_MODEL, test_set_x.get_value()[:k])
labels = get_labels()[:k]

print("Prediction for first %i samples:" % (k))
print(prediction)
print("Labels for first %i samples:" % (k))
print(labels)

... loading data
Prediction for first 10 samples:
[7 2 1 0 4 1 4 9 6 9]
Labels for first 10 samples:
[7 2 1 0 4 1 4 9 5 9]
